In [2]:
import pandas as pd
import numpy as np
import re

In [15]:
# Define the log file path
log_file = r""
output_path = r""

In [ ]:
# Prepare regex patterns for each field
block_start = re.compile(r"Decoded output:", re.MULTILINE)
decoded_output = re.compile(r"```json\s+({.*?})\s+```", re.DOTALL)
tokenization = re.compile(
    r"Tokenization time: ([\d.]+)s \| Inference time: ([\d.]+)s \| Input Tokens: (\d+) \| Output Tokens: (\d+)"
)
prob_label = re.compile(r"Predicted probability: ([\d.]+) \| True label: (\d+)")
diagnosis = re.compile(r"Diagnosis for: (.+)")
explanation = re.compile(r"Generated explanation: (.+)")
system_message = re.compile(r"System message: (.+)")
system_message_idx = re.compile(r"System message index: (\d+)")

# Read the log file
with open(log_file, "r", encoding="utf-8") as f:
    log = f.read()


# Only process after the start line

start_line = "Starting test evaluation..."

start_idx = log.find(start_line)

if start_idx != -1:

    log = log[start_idx + len(start_line) :]

In [10]:
# Split into blocks
blocks = re.split(block_start, log)
blocks = blocks[1:]  # Skip the first empty block
rows = []

In [11]:
for block in blocks:
    if not block.strip():
        continue

    # Extract fields
    decoded = decoded_output.search(block)
    diagnosis_json = None
    if decoded:
        import json

        try:
            diagnosis_json = json.loads(decoded.group(1))
        except Exception:
            diagnosis_json = {}

    token_match = tokenization.search(block)
    prob_label_match = prob_label.search(block)
    diagnosis_match = diagnosis.search(block)
    explanation_match = explanation.search(block)
    system_msg_match = system_message.search(block)
    system_msg_idx_match = system_message_idx.search(block)

    row = {
        "Input Prompt": "",  # Not present in your sample, fill as needed
        "Target Label": prob_label_match.group(2) if prob_label_match else "",
        "Predicted Probability": prob_label_match.group(1) if prob_label_match else "",
        "Predicted Diagnosis": (
            diagnosis_json.get("diagnosis", "") if diagnosis_json else ""
        ),
        "Predicted Explanation": (
            diagnosis_json.get("explanation", "") if diagnosis_json else ""
        ),
        "Tokenization Time": token_match.group(1) if token_match else "",
        "Inference Time": token_match.group(2) if token_match else "",
        "Input Tokens": token_match.group(3) if token_match else "",
        "Output Tokens": token_match.group(4) if token_match else "",
        "System Message": system_msg_match.group(1) if system_msg_match else "",
        "System Message Index": (
            system_msg_idx_match.group(1) if system_msg_idx_match else ""
        ),
    }
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(
    rows,
    columns=[
        "Input Prompt",
        "Target Label",
        "Predicted Probability",
        "Predicted Diagnosis",
        "Predicted Explanation",
        "Tokenization Time",
        "Inference Time",
        "Input Tokens",
        "Output Tokens",
        "System Message",
        "System Message Index",
    ],
)

In [12]:
df["System Message Index"] = np.tile(np.arange(5), len(df))[:len(df)]
df

,Input Prompt,Target Label,Predicted Probability,Predicted Diagnosis,Predicted Explanation,Tokenization Time,Inference Time,Input Tokens,Output Tokens,System Message,System Message Index
0,,0,0.95,diagnosis,The patient exhibits multiple concerning signs...,0.0118,102.9523,3824,300,,0
1,,0,0.95,sepsis,The patient exhibits multiple signs consistent...,0.0044,100.6231,4019,300,,1
2,,0,0.95,sepsis,The patient exhibits multiple concerning signs...,0.0044,43.0694,4114,300,,2
3,,0,0.95,sepsis,The patient exhibits multiple concerning signs...,0.0038,43.2931,4146,300,,3
4,,0,0.95,sepsis,The patient exhibits multiple concerning signs...,0.0041,43.0348,4222,300,,4
...,...,...,...,...,...,...,...,...,...,...,...
435,,0,0.85,diagnosis,The patient exhibits several concerning signs ...,0.0034,8.2341,3803,300,,0
436,,0,0.78,sepsis,Patient exhibits concerning signs suggestive o...,0.0036,8.2341,3998,300,,1
437,,0,0.78,sepsis,Patient exhibits concerning signs for sepsis. ...,0.0037,8.3317,4093,300,,2
438,,0,0.78,sepsis,Patient exhibits concerning signs for sepsis. ...,0.0040,8.3617,4125,300,,3


In [ ]:
df.to_csv(
    output_path,
    index=False,
)